In [1]:
import pandas as pd 
import numpy as np 
import requests 
import time 
import bs4
from bs4 import BeautifulSoup
import random
import re
from gensim import corpora, models, similarities
import nltk
from nltk.corpus import stopwords
import sys

In [75]:
#UPDATE TOKEN!! - I use the 'Get Cookies' extension
session_id = '133-1288454-0410613'
tolkien = 'JH4ACFPFnQmzw54dUa2l5UoL9UkfAlRkwxLJ05kw9QQsRaMmAug8Rtjx04s1hPEf3f2aosCZJ8H+DbZ7M5YLCQqInaSHC0w0QY7loYF8QxwfelJ7oVJsV5G6f7Rres9XjDiSomlX+MXHj08ZekTm5bfbWnsfp2FmSu7dQaiDpBPdxbRwSp0flsAS/rjDWd6XxoEAfsxvy0XSep3MiuCtArrPAKm7Kamv'
print('type search term: whatever term that pulls up the amzn product you are looking to analyze')
search1 = input()
print('type the first 2+ words of the product that you would like to analyze')
print('AS IT appears on amazon - exact letters: dewalt drill set --> "dewalt 20v max"')
productID = input()
caregory = 'air fryer'
key_word = search1

reviewhigh = 'five'
rh = '5'
reviewlow = 'one'
rl = '1'


type search term: whatever term that pulls up the amzn product you are looking to analyze
x box
type the first 2+ words of the product that you would like to analyze
AS IT appears on amazon - exact letters: dewalt drill set --> "dewalt 20v max"
xbox series x


In [ ]:
search = search1.lower().replace(' ','+')
productID = productID.lower()
lenPID = int(len(productID))
url = 'https://www.amazon.com/s?k='+search
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36'}
cookies = {'session-id':session_id,'session-id-time':'2082787201l','i18n-prefs':'USD','skin':'noskin','ubid-main':'134-2453728-6483934','csm-hit':'YQQQW2SST3FEWRY0ACP5|1659552174494&t:1659552177027&adb:adblk_no','ubid-main':'132-0411939-8328406','session-token':tolkien}
html = requests.get(url,headers = headers, cookies =cookies).text
soup = bs4.BeautifulSoup(html, "html.parser")
xc = str(soup).split('a-text-normal">')
for xx in range(len(xc)):
  print(xc[xx][:lenPID].lower())
  if xc[xx][:lenPID].lower() == productID:
    try:
      x = 0
      if x<=1:
          price = xc[xx].split('<span class="a-offscreen">')[1].split('<')[0].replace('$','')
          product = xc[xx].split('<')[0]
          reviewAvg = xc[xx].split('<div class="a-row a-size-small"><span aria-label="')[1].split('">')[0].split()[0]
          reviewCount = xc[xx].split('</span><span aria-label="')[1].split('"')[0]
          link = 'https://www.amazon.com' + xc[xx-1].split('href="')[-1].split('"')[0]
          x = 2
      else:
        dfd = 0
    except:
      fdf = 0
  else:
    fdf = 0

df = {'product':[],'reviewAvg':[],'reviewCount':[],'price':[],'link':[]}
df['product'].append(product)
df['reviewAvg'].append(reviewAvg)
df['reviewCount'].append(reviewCount)
df['price'].append(price)
df['link'].append(link)

pid = str(link.split('dp%2F')[1].split('%2')[0])
hightext = []
lowtext = []
qtext = []

for ux in range(40):
 try:
  high_review_url = 'https://www.amazon.com/product-reviews/'+pid+'/ref=acr_dp_hist_'+rh+'?ie=UTF8&filterByStar='+reviewhigh+'_star&reviewerType=all_reviews&pageNumber='+str(ux)+'#reviews-filter-bar'
  low_review_url = 'https://www.amazon.com/product-reviews/'+pid+'/ref=acr_dp_hist_'+rl+'?ie=UTF8&filterByStar='+reviewlow+'_star&reviewerType=all_reviews&pageNumber='+str(ux)+'#reviews-filter-bar'
  question_url = 'https://www.amazon.com/ask/questions/asin/'+pid+'/'+str(ux)+'/ref=ask_dp_iaw_ql_hza?'
  url = high_review_url
  ht = []
  html = requests.get(url,headers = headers, cookies =cookies).text
  soup = bs4.BeautifulSoup(html, "html.parser")
  xc = str(soup).split('Reviewed in the United States')
  for xi in range(len(xc)-1):
    try:
      ht.append(xc[xi].split('<span>')[1].split('</span>')[0])
    except:
      fdf = 0
  for xu in range(len(ht)):
    if int(len(ht[xu].split('{"')))>=2:
      fdf = 0
    else:
      hightext.append(ht[xu].replace("\'","").replace('<br/>','').replace('\xa0','').replace('(','').replace(')',''))
  time.sleep(5)
  url = low_review_url
  lt = []
  html = requests.get(url,headers = headers, cookies =cookies).text
  soup = bs4.BeautifulSoup(html, "html.parser")
  xc = str(soup).split('Reviewed in the United States')
  for xi in range(len(xc)-1):
    try:
      lt.append(xc[xi].split('<span>')[1].split('</span>')[0])
    except:
      fdf = 0
  for xu in range(len(lt)):
    if int(len(lt[xu].split('{"')))>=2:
      fdf = 0
    else:
      lowtext.append(lt[xu].replace("\'","").replace('<br/>','').replace('\xa0','').replace('(','').replace(')',''))
  time.sleep(random.uniform(0,9))
  url = question_url
  html = requests.get(url,headers = headers, cookies =cookies).text
  soup = bs4.BeautifulSoup(html, "html.parser")
  xc = str(soup).split('Question:')
  qt=[]
  for xj in range(len(xc)):
    qt.append(xc[xj].split('type="widget">')[1].split('</')[0].replace('\n                ','').replace('<br/>','').replace('\xa0','').replace('(','').replace(')',''))
  for xw in range(len(qt)):
    if int(len(str(qt[xw]).split('<')))>=2:
      fdf = 0
    else:
      qtext.append(qt[xw].replace('  ','').replace('\n',''))
  time.sleep(5)
 except:
    print('yeet')
f5star = []
for ui in range(len(hightext)):
    hh = hightext[ui].split('.')
    for xi in range(len(hh)):
        f5star.append(hh[xi].replace('</br>','').replace('<br>',''))
f5star
fiveStar = pd.DataFrame()
fiveStar['text'] = f5star
fiveStar = fiveStar.drop_duplicates()
fiveStar.to_csv('fiveStarReviews.csv')

o1star = []
for ui in range(len(lowtext)):
    hh = lowtext[ui].split('.')
    for xi in range(len(hh)):
        o1star.append(hh[xi].replace('</br>','').replace('<br>',''))

oneStar = pd.DataFrame()
oneStar['text'] = o1star
oneStar = oneStar.drop_duplicates()
oneStar.to_csv('oneStarReviews.csv')
        
questions = pd.DataFrame()
questions['text'] = qtext
questions.to_csv('questions.csv')

fStar = fiveStar
oStar = oneStar
qtons = questions

<!doctype htm

<div class="
results</span
dabao securit
xbox series s
nov 10, 2020<
xbox series s
dec 10, 2021<
xbox series x
nov 10, 2020<
microsoft xbo
jan 14, 2019<
xbox one 500g
microsoft xbo
microsoft xbo
xbox one s 1t
microsoft - x
2021 microsof
xbox one s (r
more results<
microsoft xbo
xbox live gol
nov 15, 2005<
click to see 
microsoft xbo
microsoft xbo
jan 14, 2019<
minecraft dun
sep 8, 2020</
xbox series s
seagate stora
anime x-box s
more results<
xbox core wir
nov 10, 2020<
microsoft xbo
xbox core wir
may 11, 2021<
xbox core wir
nov 10, 2020<
smallrig rect
microsoft - x
$60 xbox gift
sep 6, 2013</
8k hdmi switc
hyperx cloud 
brady m71ep-1


In [ ]:
#scrapyard, could not get en_core_web_sm to load so I used google colab - here's the link!
#colab link: https://colab.research.google.com/drive/1nXoDkhikJHkwXeEqMJyhqLQwaPV4TTeL?usp=sharing

In [ ]:
#scrapyard - same function that is in google colab!

In [63]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import pandas as pd
import numpy as np  


from pandas.core.arrays.boolean import BooleanDtype

def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', str(sent))  
        sent = re.sub('\s+', ' ', str(sent))  
        sent = re.sub("\'", "", str(sent))  
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  
        
def cleaner(df):
  data = df.text.values.tolist()
  data_words = list(sent_to_words(data))
  return data_words, data
print('*')
print('**')
print('***')
print('Enter high or low, to analyze 5 star vs 1 star reviews')
print('Enter the number of topics you would like to analyze (rec: 5-10)')
print('***')
print('**')
print('*')

def topicMaster(target,num_topics, savedAs):
  if target.lower() == 'high':
    target = fStar
  elif target.lower() == 'low':
    target = oStar
  else:
    target = fStar
    print('high/low not entered, default: 5 star reviews')
  data_words, data = cleaner(target)
  bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
  trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)

  def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
      """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
      texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
      texts = [bigram_mod[doc] for doc in texts]
      texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
      texts_out = []
      nlp = spacy.load('en_core_web_sm')
      for sent in texts:
          doc = nlp(" ".join(sent)) 
          texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
      texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
      return texts_out
  data_ready = process_words(data_words)
  id2word = corpora.Dictionary(data_ready)

  corpus = [id2word.doc2bow(text) for text in data_ready]
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=num_topics, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=10,
                                            passes=10,
                                            alpha='symmetric',
                                            iterations=100,
                                            per_word_topics=True)

  pprint(lda_model.print_topics())

  def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
      sent_topics_df = pd.DataFrame()
      for i, row_list in enumerate(ldamodel[corpus]):
          row = row_list[0] if ldamodel.per_word_topics else row_list            
          row = sorted(row, key=lambda x: (x[1]), reverse=True)
          for j, (topic_num, prop_topic) in enumerate(row):
              if j == 0: 
                  wp = ldamodel.show_topic(topic_num)
                  topic_keywords = ", ".join([word for word, prop in wp])
                  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
              else:
                  break
      sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
      sent_topics_df=sent_topics_df[sent_topics_df['Dominant_Topic']>=.5]
      contents = pd.Series(texts)
      sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
      return(sent_topics_df)

  df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)
  df_dominant_topic = df_topic_sents_keywords.reset_index()
  df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

  pd.options.display.max_colwidth = 100
  sent_topics_sorteddf_mallet = pd.DataFrame()
  sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')
  for i, grp in sent_topics_outdf_grpd:
      sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                              grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                              axis=0)
  sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)
  sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

  import pyLDAvis
  import pyLDAvis.gensim_models
  pyLDAvis.enable_notebook()
  vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary=lda_model.id2word)
  p = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary=lda_model.id2word)
  pyLDAvis.save_html(p, savedAs)
  return vis

topicMaster(input(),int(input()), 'fStar.html')

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [49]:
qtext

["Why is it Amazon can never deliver on the day promised? Always 2 to 3 days late. I'd be better off going to a regular store",
 'Why did the cost go from $9.21 in April 2022 when I last ordered these to $22.39 now.? This is more than a 100% increase!',
 'Is this product non drowsy',
 'What is the Expiration date ? ',
 'Where arw these made?',
 'Where are the labs credentials/certifications?',
 'Does this have a taste?',
 'what is the difference between this and the RX brand',
 'Why did this double in price in the last month?',
 "Why did the price go up so much recently? This is laughable and I won't buy this particular item again. ",
 "Why is it Amazon can never deliver on the day promised? Always 2 to 3 days late. I'd be better off going to a regular store",
 'Why did the cost go from $9.21 in April 2022 when I last ordered these to $22.39 now.? This is more than a 100% increase!',
 'Is this product non drowsy',
 'What is the Expiration date ? ',
 'Where arw these made?',
 'Where are

In [66]:
pip install pyLDAvis

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136882 sha256=a167e60bedb922e40156f44d910bf2de17ffdf4dd93f8fc8ed9bca7f656e8288
  Stored in directory: c:\users\w7car\appdata\local\pip\cache\wheels\57\a4\86\d10c6c2e0bf149fbc0afb0aa5a6528ac35b30a133a0270c477
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=5781341e610eaf3291ae07ea545b9b37e1ba421082f23ae0e9921e7265de57a7
  Stored in directory: c:\users\w7car\appdata\local\pip\cache\wheels\e4\7b\98\b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully

In [71]:
import en_core_web_sm

spacy.load('en_core_web_sm')

ModuleNotFoundError: No module named 'en_core_web_sm'

In [70]:
pip install en_core_web

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement en_core_web (from versions: none)
ERROR: No matching distribution found for en_core_web


In [21]:
link

'https://www.amazon.com/sspa/click?ie=UTF8&amp;spc=MTo1Njg0Mjg5MzQ3MTIyNjE5OjE2NjA5MjA3NzQ6c3BfYnRmOjIwMDAzNDA5Njk1Njk3MTo6MDo6&amp;sp_csd=d2lkZ2V0TmFtZT1zcF9idGY&amp;url=%2FAmazon-Basic-Care-Levocetirizine-Dihydrochloride%2Fdp%2FB083VVT61S%2Fref%3Dsr_1_58_sspa%3Fkeywords%3Dallergy%2Bmedication%26qid%3D1660920774%26sr%3D8-58-spons%26psc%3D1'

In [10]:
soup

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-us"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="https://images-na.ssl-images-amazon.com" rel="dns-prefetch"/>
<link href="https://m.media-amazon.com" rel="dns-prefetch"/>
<link href="https://completion.amazon.com" rel="dns-prefetch"/>
<!-- sp:end-feature:cs-optimization -->
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.

In [9]:
url

'https://www.amazon.com/s?k=allergy+medication'

In [16]:
productID

'amazon basic care'

In [ ]:
url = question_url
html = requests.get(url,headers = headers, cookies =cookies).text
soup = bs4.BeautifulSoup(html, "html.parser")
xc = str(soup).split('Question:')
qt=[]
for xj in range(len(xc)):
  qt.append(xc[xj].split('type="widget">')[1].split('</')[0].replace('\n                ','').replace('<br/>','').replace('\xa0','').replace('(','').replace(')',''))
for xw in range(len(qt)):
  if int(len(str(qt[xw]).split('<')))>=2:
    fdf = 0
  else:
    qtext.append(qt[xw].replace('  ','').replace('\n',''))




qtext

['would a 6 inch by 3 inch baking pan work in this? ',
 'I am looking for one with automatic temperature controlsettingsi.e. fish--french fries etc.Does this one have that?',
 'My basket handle broke, is there a place to buy just replacement parts?',
 'Is it loud.',
 'does it work in 240 voltage ?',
 'What is its capacity?',
 'Can you cook cheese sticks with this?',
 'Does it heat up the kitchen?',
 'Did the food taste fried or does it taste like it came from the oven?',
 'How to buy just the basket for this product ']

In [ ]:
qtext[1].replace('  ','').replace('\n','')

'I am looking for one with automatic temperature controlsettingsi.e. fish--french fries etc.Does this one have that?'

In [ ]:
qtext[0].split('<')

2

In [ ]:
question_url

'https://www.amazon.com/ask/questions/asin/B01DIQ817G/2/ref=ask_dp_iaw_ql_hza?'

In [ ]:
url = high_review_url
html = requests.get(url,headers = headers, cookies =cookies).text
soup = bs4.BeautifulSoup(html, "html.parser")
xc = str(soup).split('Reviewed in the United States')
hightext = []
for xi in range(len(xc)-1):
  try:
    hightext.append(xc[xi].split('<span>')[1].split('</span>')[0])
  except:
    fdf = 0
#'Reviewed in the United States' > '<span>' + '</span>'
hightext
#<br/> + \'

['Prime',
 'Listen to only this review. Forget everything else you read.',
 "I am here to clear the air with this air fryer.<br/><br/>Anyone giving it a 1 star either has a major issue that has nothing to do with the quality of the product or they are the pickiest person ever.<br/><br/>Look it's 65 dollars as of this review. I had my air fryer since Dec 8th 2020.<br/><br/>I am reading these reviews like omg rolling my eyes. Ok there is only one temperature you are ever going to use it on and that is max 400. There is no point in using the other temps as they do nothing but cook the food slower. Yes the numbers are small but who cares, turn the knob all the way until it doesn't move anymore and leave it there forever. The other knob is just the timer on how long its going to cook. Who cares reading the numbers just crank it and come back every 5 min and check on it. That is it. That simple to use. There is no need to read the dial once you know how to use the machine correctly.<br/><br/

In [ ]:
ht = hightext
hightext = []
for xu in range(len(ht)):
  if int(len(ht[xu].split('{"')))>=2:
    fdf = 0
  else:
    hightext.append(ht[xu].replace("\'","").replace('<br/>','').replace('\xa0','').replace('(','').replace(')',''))


['Prime',
 'Listen to only this review. Forget everything else you read.',
 'I am here to clear the air with this air fryer.Anyone giving it a 1 star either has a major issue that has nothing to do with the quality of the product or they are the pickiest person ever.Look its 65 dollars as of this review. I had my air fryer since Dec 8th 2020.I am reading these reviews like omg rolling my eyes. Ok there is only one temperature you are ever going to use it on and that is max 400. There is no point in using the other temps as they do nothing but cook the food slower. Yes the numbers are small but who cares, turn the knob all the way until it doesnt move anymore and leave it there forever. The other knob is just the timer on how long its going to cook. Who cares reading the numbers just crank it and come back every 5 min and check on it. That is it. That simple to use. There is no need to read the dial once you know how to use the machine correctly.This air fryer can cook food quickly. I l

In [ ]:
ht = hightext
hightext = []
for xu in range(len(ht)):
  if int(len(ht[xu].split('{"')))>=2:
    fdf = 0
  else:
    hightext.append(ht[xu])
hightext

hightext

In [ ]:
link.split('dp/')[1].split('/')[0]

'B01DIQ817G'

In [ ]:
str(soup)

'<!DOCTYPE doctype html>\n<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-us"><!-- sp:feature:head-start -->\n<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>\n<!-- sp:end-feature:head-start -->\n<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>\n<!-- sp:feature:cs-optimization -->\n<meta content="on" http-equiv="x-dns-prefetch-control"/>\n<link href="https://images-na.ssl-images-amazon.com" rel="dns-prefetch"/>\n<link href="https://m.media-amazon.com" rel="dns-prefetch"/>\n<link href="https://completion.amazon.com" rel="dns-prefetch"/>\n<!-- sp:end-feature:cs-optimization -->\n<script type="text/javascript">\nwindow.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\nif (window.ue_ihb === 1) {\n\nvar ue_csm = window,\n    ue_hob = +new Date();\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([

In [ ]:
reviewCount

'2,003'

In [ ]:
xc[xx].split('<span class="a-offscreen">')

['BLACK+DECKER Purify 2-Liter Air Fryer, White/Black, HF100WD</span> </a> </h2></div><div class="a-section a-spacing-none a-spacing-top-micro"><div class="a-row a-size-small"><span aria-label="4.6 out of 5 stars"><span class="a-declarative" data-a-popover=\'{"closeButton":false,"closeButtonLabel":"","position":"triggerBottom","popoverLabel":"","url":"/review/widgets/average-customer-review/popover/ref=acr_search__popover?ie=UTF8&amp;asin=B00NPYJW2A&amp;ref=acr_search__popover&amp;contextId=search"}\' data-action="a-popover" data-csa-c-func-deps="aui-da-a-popover" data-csa-c-type="widget"><a class="a-popover-trigger a-declarative" href="javascript:void(0)" role="button"><i class="a-icon a-icon-star-small a-star-small-4-5 aok-align-bottom"><span class="a-icon-alt">4.6 out of 5 stars</span></i><i class="a-icon a-icon-popover"></i></a></span> </span><span aria-label="980"><a class="a-link-normal s-underline-text s-underline-link-text s-link-style" href="/DECKER-Purify-2-Liter-Black-HF100WD

In [ ]:
xc[xx].split('<span class="a-offscreen">')[1].split('<')[0]

['black+decker', 'purify', '2-liter', 'air', 'fryer']

In [ ]:
xc

'Magic Bullet MBA50100 Air Fryer</span> </a> </h2></div><div class="a-section a-spacing-none a-spacing-top-micro"><div class="a-row a-size-small"><span aria-label="4.5 out of 5 stars"><span class="a-declarative" data-a-popover=\'{"closeButton":false,"closeButtonLabel":"","position":"triggerBottom","popoverLabel":"","url":"/review/widgets/average-customer-review/popover/ref=acr_search__popover?ie=UTF8&amp;asin=B0923CBH72&amp;ref=acr_search__popover&amp;contextId=search"}\' data-action="a-popover" data-csa-c-func-deps="aui-da-a-popover" data-csa-c-type="widget"><a class="a-popover-trigger a-declarative" href="javascript:void(0)" role="button"><i class="a-icon a-icon-star-small a-star-small-4-5 aok-align-bottom"><span class="a-icon-alt">4.5 out of 5 stars</span></i><i class="a-icon a-icon-popover"></i></a></span> </span><span aria-label="273"><a class="a-link-normal s-underline-text s-underline-link-text s-link-style" href="/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie

In [ ]:
xc[2][:lenPID].lower()==productID

True

In [ ]:
productID

'black+decker purify 2-liter air fryer'